In [1]:
# create and pickle STUMS curated subset of PUMS.
# https://www2.census.gov/programs-surveys/acs/data/pums/2022/1-Year/downloads
# process the original dataframe

import pandas as pd
import numpy as np
import socket

dire = 'D:\\Dropbox\\Census\\Census2024\\FilesToGitHub\\'
pfile = 'psam_ptx.csv'

DOFILE = pfile
DECILATE = True # below, replace values by deciles in selected columns
REMAPILATE = True # below, use bespoke remappings

def replace_all_by_deciles(dff, column_name): # neg and pos vals both meaningful, 1-10
    fiddle = dff[column_name] + np.random.uniform(0, 0.01, size=len(dff))
    decile_bins = np.percentile(fiddle, np.arange(10, 100, 10))
    deciles = np.digitize(fiddle, bins=decile_bins, right=True) + 1
    dff[column_name] = deciles.astype(float)
def replace_pos_by_deciles(dff, column_name): # 1-10 leaving neg or zero unchanged
    positive_indices = dff[column_name] > 0.
    positive_values = dff.loc[positive_indices, column_name]
    positive_values += np.random.uniform(0, 0.01, size=len(positive_values))
    decile_bins = np.percentile(positive_values, np.arange(10, 100, 10))
    deciles = np.digitize(positive_values, bins=decile_bins, right=True) + 1
    dff.loc[positive_indices, column_name] = deciles.astype(float)
def make_dict_of_colnams(file_path):
    result_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            words = line.split()
            if words:  # Ensure there are words to avoid index errors
                first_word = words[0]
                if first_word.isupper() and ': ' in line:
                    key, value = line.split(': ', 1)
                    result_dict[key] = value
    return result_dict
def map_values(array, instructions): # example: "-1:7, 1-4:8, 6:-1, 8-10:19"
    mapping_dict = {}
    for part in instructions.split(','):
        key, value = part.strip().split(':')
        # Trim whitespace and convert the value to integer
        value = float(value.strip())
        if '-' in key and not key.startswith('-'):  # Range indicator present and not a negative number
            start, end = map(int, key.split('-'))
            for i in range(start, end + 1):
                mapping_dict[i] = value
        else:
            mapping_dict[int(key)] = value
    mapped_array = [mapping_dict.get(x, x) for x in array]    
    return mapped_array


df = pd.read_csv(dire + DOFILE)
print('data before fixes:')
print(df.info())
print("")

PICKLEALL = False
if PICKLEALL:
    df.to_pickle(dire + "psam_ptx_as_df.pkl")
    print("Pickled the full original data set.")
    
SAVEWGTS = False
if SAVEWGTS :
    wgtcols = df.columns[df.columns.str.startswith(('PWGTP', 'WGTP'))]
    dfwgts = df[wgtcols]
    wgtsfile = "PWGTPweights_df.pkl"
    dfwgts.to_pickle(dire + wgtsfile)
    print(f"Pickled {wgtsfile} with this contents:")
    print(dfwgts.info())
    print("")
    
# make a dict of long descriptions of col names
colnamfile = dire + 'PUMS_Data_Dict_WHP_reduced.txt'
colnamdict = make_dict_of_colnams(colnamfile)

# data fixes: get rid of not now useful columns
dff = df.loc[:, ~df.columns.str.match('^PWGTP')] # re-sample weights, separately saved
dff = dff.loc[:, ~dff.columns.str.match('^WGTP')] # ditto
dff = dff.loc[:, ~dff.columns.str.match('^NAICSP')] # too hard, delete
dff = dff.loc[:, ~dff.columns.str.match('^SOCP')] # too hard, delete

# get rid of allocation flags
dropcols = [col for col in dff.columns if "allocation flag" in colnamdict.get(col, "")]
dff = dff.drop(columns=dropcols)

# RT is a special case with letters
dff['RT'] = dff['RT'].map({'H': 1, 'P': 2}).astype(np.int64)
# SERIALNO has letters distinguishing H from P records, not needed
dff['SERIALNO'] = dff['SERIALNO'].str.replace('\D', '', regex=True).astype(np.int64) # make numeric

# get rid of nan's
try: # will succeed for hfile, fail for pfile
    dff[['FINCP', 'HINCP']] = dff[['FINCP', 'HINCP']].fillna(0) # incomes (only cols with legit neg values)
    dff = dff[dff['TYPEHUGQ'] == 1].reset_index(drop=True) # keep only Housing Units, not Group Quarters
    dff = dff[dff['NP'] > 0].reset_index(drop=True) # remove vacant units
except:
    print('(FINCP, HINCP, TYPEHUGQ not in this dataset, no problem!)')
dff = dff.fillna(-1) # else change nans to -1
dff = dff.astype(np.float64)

if DECILATE : # reduce cardinality of answers by coding deciles on some cols
    deccols = ['PINCP','PERNP','WAGP','SEMP','INTP',] # deciles over whole range neg and os
    poscols = ['SSP','RETP','OIP','POVPIP','SSIP','PAP','JWMNP','WKHP',] # deciles over positives only
    for col in deccols: 
        if col in dff: replace_all_by_deciles(dff, col)
    for col in poscols: 
        if col in dff: replace_pos_by_deciles(dff, col)
            
if REMAPILATE : # reduce cardinality of answers by WHP crafterd mappings
    bespoke = {
        'JWAP': '1-33:1, 34-45:2, 46-57:3, 58-69:4, 70-93:5, 94-105:6, 106-117:7, 118-130:8, 131-177:9, 178-285:10',
        'JWDP': '1-33:1, 34-45:2, 46-57:3, 58-69:4, 70-93:5, 94-105:6, 106-117:7, 118-130:8, 131-177:9, 178-285:10',
        'LANP': '-1:1, 0:1, 1200:2, 1970-2050:3, 1000-1199:4, 1201-1969:4, 2051-9999:4',
        'AGEP': '1-6:1, 7-12:2, 13-19:3, 20-29:4, 30-39:5, 40-49:6, 50-59:7, 60-69:8, 70-79:9, 80-99:10',
        'YOEP': '1000-1949:1, 1950-1969:2, 1970-1989:3, 1990-1999:4, 2000-2009:5, 2010-2014:6, 2015-2019:7, 2020-9999:8',
        'MARHYP': '1000-1949:1, 1950-1969:2, 1970-1989:3, 1990-1999:4, 2000-2009:5, 2010-2014:6, 2015-2019:7, 2020-9999:8',
        'CITWP': '1000-1949:1, 1950-1969:2, 1970-1989:3, 1990-1999:4, 2000-2009:5, 2010-2014:6, 2015-2019:7, 2020-9999:8',
        'WKWN': '1-9:1, 10-19:2, 20-29:3, 30-39:4, 40-49:5, 50-53:6',
        'SCHL': '2-6:1, 7-12:2, 13-15:3, 16-17:4, 18-19:5, 20:6, 21:7, 22-23:8, 24:9',
        'SCHG': '1-5:1, 6-11:2, 12-14:3, 15:4, 16:5',
        'HISP': '5-12:5, 13-22:6, 23-99:7',
        'VPS':  '1-5:1, 6-8:2, 9-10:3, 11:4, 12-99:5',
        'JWTRNS': '3-5:3, 9:4, 10:5, 11:6, 6-8:7, 12:7',
    }
    for col in bespoke :
        dff[col] = map_values(dff[col],bespoke[col])
dff = dff.astype(np.int64) # turn it all back to integer
            
print('data after fixes:')
print(dff.info()) # show size info
print("")

unique_counts = dff.nunique()
nuniquedict = unique_counts.to_dict()
fracnandict = {col: (dff[col] < 0).sum() / dff[col].count() for col in dff.columns}
print('done.')

data before fixes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292919 entries, 0 to 292918
Columns: 287 entries, RT to PWGTP80
dtypes: float64(84), int64(199), object(4)
memory usage: 641.4+ MB
None

(FINCP, HINCP, TYPEHUGQ not in this dataset, no problem!)
data after fixes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292919 entries, 0 to 292918
Columns: 130 entries, RT to FHINS5C
dtypes: int64(130)
memory usage: 290.5 MB
None

done.


In [2]:
# MAKE STUMS data frames:
# subset and pickle a dataframe
def makesmallerdf(dff, columns_list, n_rows):
    n_rows = min(n_rows, len(dff))
    # Randomly select rows without replacement and filter by columns
    random_rows_df = dff.sample(n=n_rows, replace=False)[columns_list].reset_index(drop=True)    
    return random_rows_df

STUMScols = ['SEX','AGEP','SCHL','RAC1P','ESR','MAR','DIS','PINCP','NATIVITY','MIL','ENG','HICOV','FER',
            'JWMNP','WAOB','SCH','SCHG','PERNP','SSP','PAP','MSP','MARHT','MARHYP','MARHW','MARHM','MARHD',
            'WRK','JWAP','WKHP','WKWN','CIT','DECADE','JWTRNS','LANX','LANP','DEYE','DEAR','DREM',]

print('There are',len(STUMScols),'STUMS columns to start with.')
sum_unique_values = sum(dff[col].nunique() for col in STUMScols) # number cols in binary (not counting weights)
print('There are',sum_unique_values,'binary STUMS columns in total.')

NSAMP = 300000
NCOLS = 100
outfile = "STUMS_df_all_foo.pkl"

dfff = makesmallerdf(dff,STUMScols[:NCOLS],NSAMP)
dfff.to_pickle(dire + outfile)
print(f"Created file with {dfff.shape}.")
print('done.')

There are 38 STUMS columns to start with.
There are 233 binary STUMS columns in total.
Created file with (292919, 38).
done.


In [3]:
%%markdown
### ABOVE HERE and BELOW HERE are separate tasks. Above is make STUMS. Below is make STUMS-H.

### ABOVE HERE and BELOW HERE are separate tasks. Above is make STUMS. Below is make STUMS-H.


In [4]:
# create and pickle STUMS-H curated subset of PUMS.
import pandas as pd
import numpy as np
import socket

dire = 'D:\\Dropbox\\Census\\Census2024\\FilesToGitHub\\'
hfile = 'psam_htx.csv'

pickleallout = "psam_htx_as_df_foo.pkl"
pickleweightsout = "PWGT_H_weights_df_foo.pkl"
reduceddictfile = 'PUMS_Data_Dict_WHP_reduced.txt' # input for both P and H records
PICKLEALL = False
SAVEWGTS = False
PRINTALL = False

DOFILE = hfile
DECILATE = True # below, replace values by deciles in selected columns
REMAPILATE = True # below, use bespoke remappings

ONLYcols = ['NP','NPF','ACR','NR','YRBLT','TEN','VALP','RMSP','HFL','WATP',
 'BLD', 'RNTP','GRNTP','MHP','MRGP','SMP','SMOCP','INSP','TAXAMT',
 'PLM','BATH','BDSP','KIT','VEH','TEL','ACCESSINET','SMARTPHONE',
 'LAPTOP','BROADBND','COMPOTHX','HISPEED','HINCP','FINCP','FS','ELEP',
 'HHT','WIF','PARTNER','HUPAC','HUGCL','CPLT','HHLDRAGEP','HHLDRRAC1P',
 'HHL','LNGI','FPARC',]

deccols = ['FINCP','HINCP',] # deciles over whole range neg and os
poscols = ['FULP','GASP','INSP','MHP','MRGP','RNTP','SMP','VALP','WATP','GRNTP',
          'SMOCP','TAXAMT'] # deciles over positives only leaving neg and zero unchanged
bespoke = {
    'NP': '7-20:7',
    'BDSP': '6-99:6',
    'ELEP': '2-19:2, 20-49:3, 50-99:4, 100-199:5, 200-399:6, 400-999:7, 1000-9999:8',
    'RMSP': '4-6:4, 6-10:5, 10-99:6',
    'YRBLT': '1000-1939:1, 1940-1959:1, 1960-1979:2, 1980-1999:3, 2000-2019:4, 2020-2099:5',
    'HHLDRAGEP': '15-19:1, 20-29:2, 30-39:3, 40-49:4, 50-59:5, 60-69:6, 70-79:7, 80-99:8',
    'HHLDRRAC1P': '3-5:3, 6-6:4, 7-7:3, 8-9:5',
    'NPF': '6-8:6, 9-12:7, 13-99:8',    
}

def replace_all_by_deciles(dff, column_name): # neg and pos vals both meaningful, 1-10
    fiddle = dff[column_name] + np.random.uniform(0, 0.01, size=len(dff))
    decile_bins = np.percentile(fiddle, np.arange(10, 100, 10))
    deciles = np.digitize(fiddle, bins=decile_bins, right=True) + 1
    dff[column_name] = deciles.astype(float)
def replace_pos_by_deciles(dff, column_name): # 1-10 leaving neg or zero unchanged
    positive_indices = dff[column_name] > 0.
    positive_values = dff.loc[positive_indices, column_name]
    positive_values += np.random.uniform(0, 0.01, size=len(positive_values))
    decile_bins = np.percentile(positive_values, np.arange(10, 100, 10))
    deciles = np.digitize(positive_values, bins=decile_bins, right=True) + 1
    dff.loc[positive_indices, column_name] = deciles.astype(float)
def make_dict_of_colnams(file_path):
    result_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            words = line.split()
            if words:  # Ensure there are words to avoid index errors
                first_word = words[0]
                if first_word.isupper() and ': ' in line:
                    key, value = line.split(': ', 1)
                    result_dict[key] = value
    return result_dict
def map_values(array, instructions): # example: "-1:7, 1-4:8, 6:-1, 8-10:19"
    mapping_dict = {}
    for part in instructions.split(','):
        key, value = part.strip().split(':')
        # Trim whitespace and convert the value to integer
        value = float(value.strip())
        if '-' in key and not key.startswith('-'):  # Range indicator present and not a negative number
            start, end = map(int, key.split('-'))
            for i in range(start, end + 1):
                mapping_dict[i] = value
        else:
            mapping_dict[int(key)] = value
    mapped_array = [mapping_dict.get(x, x) for x in array]    
    return mapped_array


df = pd.read_csv(dire + DOFILE)
if PRINTALL :
    print('data before fixes:')
    print(df.info())
    print("")

if PICKLEALL:
    df.to_pickle(dire + pickleallout)
    print("Pickled the full original data set.")
    
if SAVEWGTS :
    wgtcols = df.columns[df.columns.str.startswith(('PWGTP', 'WGTP'))]
    dfwgts = df[wgtcols]
    wgtsfile = pickleweightsout
    dfwgts.to_pickle(dire + wgtsfile)
    if PRINTALL :
        print(f"Pickled {wgtsfile} with this contents:")
        print(dfwgts.info())
        print("")
    
# make a dict of long descriptions of col names
colnamfile = dire + reduceddictfile
colnamdict = make_dict_of_colnams(colnamfile)


# data fixes: get rid of not now useful columns
dff = df.loc[:,ONLYcols]
dff = dff.loc[:, ~dff.columns.str.match('^PWGTP')] # re-sample weights, separately saved
dff = dff.loc[:, ~dff.columns.str.match('^WGTP')] # ditto
dff = dff.loc[:, ~dff.columns.str.match('^NAICSP')] # too hard, delete
dff = dff.loc[:, ~dff.columns.str.match('^SOCP')] # too hard, delete

# get rid of allocation flags
dropcols = [col for col in dff.columns if "allocation flag" in colnamdict.get(col, "")]
dff = dff.drop(columns=dropcols)

try:
    # RT is a special case with letters
    dff['RT'] = dff['RT'].map({'H': 1, 'P': 2}).astype(np.int64)
    # SERIALNO has letters distinguishing H from P records, not needed
    dff['SERIALNO'] = dff['SERIALNO'].str.replace('\D', '', regex=True).astype(np.int64) # make numeric
except :
    print('(RT, SERIALNO not in remaining dataset, no problem!)')

# get rid of nan's
try: # will succeed for hfile, fail for pfile
    dff[['FINCP', 'HINCP']] = dff[['FINCP', 'HINCP']].fillna(0) # incomes (only cols with legit neg values)
    dff = dff[dff['TYPEHUGQ'] == 1].reset_index(drop=True) # keep only Housing Units, not Group Quarters
    dff = dff[dff['NP'] > 0].reset_index(drop=True) # remove vacant units
except:
    print('(FINCP, HINCP, TYPEHUGQ not in remaining dataset, no problem!)')
dff = dff.fillna(-1) # else change nans to -1
dff = dff.astype(np.float64)

if DECILATE : # reduce cardinality of answers by coding deciles on some cols
    for col in deccols: 
        if col in dff: replace_all_by_deciles(dff, col)
    for col in poscols: 
        if col in dff: replace_pos_by_deciles(dff, col)
            
if REMAPILATE : # reduce cardinality of answers by WHP crafterd mappings
    for col in bespoke :
        dff[col] = map_values(dff[col],bespoke[col])
dff = dff.astype(np.int64) # turn it all back to integer
            
if PRINTALL :
    print('data after fixes:')
    print(dff.info()) # show size info
    print("")

unique_counts = dff.nunique()
nuniquedict = unique_counts.to_dict()
fracnandict = {col: (dff[col] < 0).sum() / dff[col].count() for col in dff.columns}
print('done.')

(RT, SERIALNO not in remaining dataset, no problem!)
(FINCP, HINCP, TYPEHUGQ not in remaining dataset, no problem!)
done.


In [5]:
# MAKE STUMS-H data frames:
# subset and pickle a dataframe
def makesmallerdf(dff, columns_list, n_rows):
    n_rows = min(n_rows, len(dff))
    # Randomly select rows without replacement and filter by columns
    if n_rows == len(dff) :
        random_rows_df = dff
    else :
        random_rows_df = dff.sample(n=n_rows, replace=False)[columns_list].reset_index(drop=True)    
    return random_rows_df

STUMSHcols = ONLYcols
print('There are',len(STUMSHcols),'STUMS-H columns to start with.')
sum_unique_values = sum(dff[col].nunique() for col in STUMSHcols) # number cols in binary (not counting weights)
print('There are',sum_unique_values,'binary STUMS-H columns in total.')

outfile = "STUMS-H_df_all_foo.pkl"

dfff =  dff #makesmallerdf(dff,STUMScols[:NCOLS],NSAMP)
dfff.to_pickle(dire + outfile)
print(f"Created file {outfile} with {dfff.shape}.")
print('done.')

There are 46 STUMS-H columns to start with.
There are 315 binary STUMS-H columns in total.
Created file STUMS-H_df_all_foo.pkl with (133016, 46).
done.


In [6]:
# one-time get reduced version of data dictionary
# this, with some hand-editing, produces the file PUMS_Data_Dict_WHP_reduced.txt
dire = 'D:\\Dropbox\\Census\\Census2024\\FilesToGitHub\\'
filnam = 'PUMS_Data_Dictionary_2022.txt'

def process_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines into a list

    for i in range(len(lines) - 1):  # Loop through lines, stopping at the second to last to avoid index error
        words = lines[i].split()  # Split the current line into words
        if words:  # Check if the line is not empty
            first_word = words[0]  # Get the first word
            if first_word.isupper():  # Check if the first word is in all uppercase
                output_line = f"{first_word}: {lines[i+1].strip()}"  # Prepare the output line
                print(output_line) # or else change to save to a file

if False : # change to True to run
    process_file(dire+filnam)